<a href="https://colab.research.google.com/github/kaushikabhishek87/Tensorflow_projects/blob/main/08_NLP_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2021-06-27 11:30:08--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-06-27 11:30:08 (83.4 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [2]:
from helper_functions import unzip_data, compare_historys, plot_loss_curves

In [4]:
#getting data
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2021-06-27 11:31:53--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 142.250.107.128, 173.194.203.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.006s  

2021-06-27 11:31:54 (102 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [5]:
unzip_data("nlp_getting_started.zip")

In [6]:
# Becoming one with data
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
train_df["text"][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [9]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [10]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [11]:
# Class distribution
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [12]:
# Total Sample
len(train_df), len(test_df)

(7613, 3263)

In [20]:
# Visualise some random samples
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[["text","target"]][random_index:(random_index+5)].itertuples():
  _,text, target = row
  print(f"Target:{target}","(real disaster)" if target >0 else "(not real disaster)")
  print(f"Text:\n {text}\n")
  print("---\n")


Target:0 (not real disaster)
Text:
 I should probably stay away from hot glue guns.. I burned one finger pretty bad

---

Target:0 (not real disaster)
Text:
 A new type of ADHD? Head injuries in children linked to long-term attention problems http://t.co/I4FZ75Utnh

---

Target:0 (not real disaster)
Text:
 Truth...
https://t.co/h6amECX5K7
#News
#BBC
#CNN
#Islam
#Truth
#god
#ISIS
#terrorism
#Quran
#Lies http://t.co/B8iWRdxcm0

---

Target:0 (not real disaster)
Text:
 Ancient Mayan Tablet with Hieroglyphics Honors Lowly King: A 1600-year-old Mayan stone tablet describing the ... http://t.co/GLPFu0Uriz

---

Target:1 (real disaster)
Text:
 Another hour! It's August 05 2015 at 08:02PM Here's Red Rover Zombie Apocalypse 2014! http://t.co/cf9e6TU3g7 #internetradio #collegeradiÛ_

---



In [21]:
# Split data in traiing & validation dataset
from sklearn.model_selection import train_test_split

train_sentences, val_sentences, train_labels,  val_labels = train_test_split(train_df_shuffled["text"].to_numpy(), 
                                                                             train_df_shuffled["target"].to_numpy(),
                                                                             test_size=0.1,
                                                                             random_state=42
                                                                             )

In [23]:
train_sentences.shape, train_labels.shape, val_sentences.shape, val_labels.shape

((6851,), (6851,), (762,), (762,))

In [24]:
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

In [30]:
# Text Vectorization
import tensorflow as tf 
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# use default text vectorization

text_vectorization = TextVectorization(max_tokens=None,
                                standardize="lower_and_strip_punctuation",
                                split="whitespace",
                                ngrams=None,
                                output_mode="int",
                                output_sequence_length=None, #pad each seqeunce to longest sequence,
                                pad_to_max_tokens = True)


In [29]:
# Find the average number of tokens (words) in the training tweets

print(train_sentences[0].split())
round(sum([len(i.split()) for i in train_sentences ])/len(train_sentences) )

['@mogacola', '@zamtriossu', 'i', 'screamed', 'after', 'hitting', 'tweet']


15

In [36]:
## Set up tex vectorization variables
max_vocab_length = 10000 # max number of words in vocabulary
max_length = 15 # How many words from a tweet should model see

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                   output_mode="int",
                                   output_sequence_length = max_length)


In [37]:
# Fit the text vectorizer
text_vectorizer.adapt(train_sentences)

In [39]:
# Create a sample sentence & tokenize it
sample_sentence = "There's a flood in desert"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,   1,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [43]:
# Unique words in vocab
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:5], words_in_vocab[-5:]

(10000,
 ['', '[UNK]', 'the', 'a', 'in'],
 ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1'])

In [45]:
# Embeddings using embedding layer
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length,
                              output_dim=128,
                              input_length=max_length)
embedding

In [51]:
samp_embed = embedding(text_vectorizer( [sample_sentence]))

In [55]:
sample_sentence, samp_embed[0][0]

("There's a flood in desert", <tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 2.49969475e-02,  3.97772081e-02, -2.41081249e-02, -3.53679769e-02,
        -2.28692889e-02, -3.25129516e-02,  4.42219637e-02,  4.10302728e-03,
        -4.75923903e-02, -3.10884006e-02,  3.04828025e-02, -1.06350295e-02,
        -4.36556004e-02,  3.16687562e-02, -2.37147342e-02,  2.30442770e-02,
         1.81382895e-03,  4.98211645e-02,  3.46646421e-02,  1.05637088e-02,
        -4.18135300e-02,  4.80448268e-02, -2.24480778e-03,  3.68265025e-02,
         1.45943426e-02,  3.18809412e-02,  1.94481649e-02,  1.71919130e-02,
        -3.79772075e-02,  4.49487902e-02,  3.01072113e-02,  4.82735746e-02,
        -1.31830685e-02, -4.28023003e-02,  4.36920412e-02, -4.47575934e-02,
        -4.42959443e-02,  7.97461346e-03,  4.88476641e-02, -3.72470617e-02,
        -4.35432047e-03, -1.98605191e-02, -3.83848175e-02,  3.24521214e-03,
         5.27673960e-03,  3.59945633e-02,  3.55578996e-02,  4.99903038e-03,
        -4

In [57]:
### Baseline Model
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB 
from sklearn.pipeline import Pipeline 

In [59]:
model_0 = Pipeline([
                    ("tfidf",TfidfVectorizer()),
                    ("clf",MultinomialNB())
])

# Fot Pipeline to training_data
model_0.fit(train_sentences, train_labels)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [63]:
# Evaluate base line Model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Baseline accuracy = {baseline_score:.2f}")

Baseline accuracy = 0.79


In [81]:
baseline_pred = model_0.predict(val_sentences)
baseline_pred[:10]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0])

In [97]:
from sklearn.metrics import accuracy_score, precision_score, f1_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):

  """
  Calculates Models accuracy, precision, recall & return a dictionary
  """

  # accuracy  = round(accuracy_score(y_true, y_pred),2)
  # precision = round(precision_score(y_true, y_pred),2)
  # f1_score_val = round(f1_score(y_true, y_pred),2)

  model_accuracy = accuracy_score(y_true, y_pred)

  model_precision, model_recall, model_fscore , _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")

  model_results = {"accuracy": round(model_accuracy,2),
                   "precision":round(model_precision,2),
                   "recall":round(model_recall,2),
                   "f1":round(model_fscore,2)}

  return model_results

In [98]:
baseline_results = calculate_results(val_labels, baseline_pred)
baseline_results

{'accuracy': 0.79, 'f1': 0.79, 'precision': 0.81, 'recall': 0.79}

# Model 1: Feed forward neural network

In [116]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string)
X = text_vectorizer(inputs)
X = embedding(X)
# X = layers.Flatten()(X)
X = layers.GlobalAveragePooling1D()(X)
outputs = layers.Dense(1,activation="sigmoid")(X)

model_1 = tf.keras.Model(inputs, outputs, name="model_1_desnse")

model_1.summary()


Model: "model_1_desnse"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_4 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
global_average_pooling1d_2 ( (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 129       
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [117]:
model_1.compile(loss="binary_crossentropy",
                optimizer="Adam",
                metrics=["accuracy"])

model_1_history = model_1.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels))

Epoch 1/5
215/215 [==============================] - 4s 17ms/step - loss: 0.4829 - accuracy: 0.8551 - val_loss: 0.4830 - val_accuracy: 0.7874
Epoch 2/5
215/215 [==============================] - 4s 17ms/step - loss: 0.3135 - accuracy: 0.8978 - val_loss: 0.4598 - val_accuracy: 0.7953
Epoch 3/5
215/215 [==============================] - 4s 17ms/step - loss: 0.2501 - accuracy: 0.9150 - val_loss: 0.4679 - val_accuracy: 0.7874
Epoch 4/5
215/215 [==============================] - 4s 17ms/step - loss: 0.2119 - accuracy: 0.9279 - val_loss: 0.4842 - val_accuracy: 0.7848
Epoch 5/5
215/215 [==============================] - 3s 15ms/step - loss: 0.1836 - accuracy: 0.9390 - val_loss: 0.5115 - val_accuracy: 0.7887


In [118]:
model_1_pred_probs = model_1.predict(val_sentences)

In [120]:
model_1_pred_probs[0]

array([0.320305], dtype=float32)

In [126]:
model_1_pred = tf.squeeze(tf.round(model_1_pred_probs))
model_1_pred[0:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [127]:
model_1_results = calculate_results(val_labels, model_1_pred)
model_1_results

{'accuracy': 0.79, 'f1': 0.79, 'precision': 0.79, 'recall': 0.79}

In [128]:
import numpy as np
np.array(list(model_1_results)) > np.array(list(baseline_results))

array([False, False, False, False])

In [134]:
### Get the weight matric of embedding layer , vector of each word/token from vocabulary
embed_weights = model_1.get_layer("embedding").get_weights()[0]

In [137]:
embed_weights.shape # length same as total vocabulary

(10000, 128)

In [140]:
# Visualizing vectors in Tensorflow projector


#####Skipped - public view